In [3]:
import os
import torch
import torchvision
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

class CIFARClassificationDataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=32, num_workers=4):
        """
        CRISP-DM - Data Understanding and Preparation Phase
        """
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        # Data transformation pipeline
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # VGG expects 224x224 images
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])  # ImageNet stats
        ])

    def prepare_data(self):
        """Download datasets if not available"""
        torchvision.datasets.CIFAR10(root=self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR10(root=self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        """Split data into train, validation, and test sets"""
        full_dataset = torchvision.datasets.CIFAR10(
            root=self.data_dir, train=True, transform=self.transform
        )
        test_dataset = torchvision.datasets.CIFAR10(
            root=self.data_dir, train=False, transform=self.transform
        )

        # 80-20 train-validation split
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        self.train_dataset, self.val_dataset = random_split(
            full_dataset, [train_size, val_size]
        )
        self.test_dataset = test_dataset

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )

class VGG19Classifier(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=1e-3):
        """
        CRISP-DM - Modeling Phase
        Configure VGG19 for transfer learning
        """
        super().__init__()
        self.learning_rate = learning_rate

        # Load pre-trained VGG19
        self.model = models.vgg19(pretrained=True)

        # Freeze base model parameters
        for param in self.model.parameters():
            param.requires_grad = False

        # Replace classifier head
        num_features = self.model.classifier[6].in_features
        features = list(self.model.classifier.children())[:-1]
        features.extend([
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        ])
        self.model.classifier = nn.Sequential(*features)

        # Loss and metrics
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).float() / len(y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).float() / len(y)
        self.log('test_accuracy', acc)
        return acc

    def configure_optimizers(self):
        """
        CRISP-DM - Modeling Phase: Optimizer Configuration
        """
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=3
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

def main():
    """
    CRISP-DM - Deployment Phase
    """
    # Set random seed for reproducibility
    pl.seed_everything(42)

    # Data Module
    dm = CIFARClassificationDataModule()

    # Model
    model = VGG19Classifier()

    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor='val_accuracy',
        mode='max',
        save_top_k=3,
        filename='vgg19-{epoch:02d}-{val_accuracy:.2f}'
    )

    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=True,
        mode='min'
    )

    # Trainer
    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1 if torch.cuda.is_available() else 0,
        callbacks=[checkpoint_callback, early_stop_callback],
        precision=16 if torch.cuda.is_available() else 32
    )

    # Train
    trainer.fit(model, dm)

    # Test
    trainer.test(model, dm)

if __name__ == "__main__":
    main()

INFO:lightning_fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 164MB/s]


TypeError: Trainer.__init__() got an unexpected keyword argument 'gpus'

In [4]:
pip install torch torchvision

In [2]:
pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 46.9 MB/s eta 0:00:00


In [6]:
import os
import torch
import torchvision
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

class CIFARClassificationDataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=32, num_workers=4):
        """
        CRISP-DM - Data Understanding and Preparation Phase
        """
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        # Data transformation pipeline
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # VGG expects 224x224 images
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])  # ImageNet stats
        ])

    def prepare_data(self):
        """Download datasets if not available"""
        torchvision.datasets.CIFAR10(root=self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR10(root=self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        """Split data into train, validation, and test sets"""
        full_dataset = torchvision.datasets.CIFAR10(
            root=self.data_dir, train=True, transform=self.transform
        )
        test_dataset = torchvision.datasets.CIFAR10(
            root=self.data_dir, train=False, transform=self.transform
        )

        # 80-20 train-validation split
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        self.train_dataset, self.val_dataset = random_split(
            full_dataset, [train_size, val_size]
        )
        self.test_dataset = test_dataset

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )

class VGG19Classifier(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=1e-3):
        """
        CRISP-DM - Modeling Phase
        Configure VGG19 for transfer learning
        """
        super().__init__()
        self.learning_rate = learning_rate

        # Load pre-trained VGG19
        self.model = models.vgg19(pretrained=True)

        # Freeze base model parameters
        for param in self.model.parameters():
            param.requires_grad = False

        # Replace classifier head
        num_features = self.model.classifier[6].in_features
        features = list(self.model.classifier.children())[:-1]
        features.extend([
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        ])
        self.model.classifier = nn.Sequential(*features)

        # Loss and metrics
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).float() / len(y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).float() / len(y)
        self.log('test_accuracy', acc)
        return acc

    def configure_optimizers(self):
        """
        CRISP-DM - Modeling Phase: Optimizer Configuration
        """
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=3
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

def main():
    """
    CRISP-DM - Deployment Phase
    """
    # Set random seed for reproducibility
    pl.seed_everything(42)

    # Data Module
    dm = CIFARClassificationDataModule()

    # Model
    model = VGG19Classifier()

    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor='val_accuracy',
        mode='max',
        save_top_k=3,
        filename='vgg19-{epoch:02d}-{val_accuracy:.2f}'
    )

    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=True,
        mode='min'
    )

    # Trainer
    trainer = pl.Trainer(
        max_epochs=10,
        accelerator='auto',  # Automatically select GPU or CPU
        devices=1,  # Use 1 device
        callbacks=[checkpoint_callback, early_stop_callback],
        precision='16-mixed' if torch.cuda.is_available() else '32-true'
    )

    # Train
    trainer.fit(model, dm)

    # Test
    trainer.test(model, dm)

if __name__ == "__main__":
    main()

INFO:lightning_fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | VGG              | 141 M  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
2.1 M     Trainable params
139 M     Non-trainable params
141 M     Total params
566.692   Total estimated model params size (MB)
52        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.004 >= min_delta = 0.0. New best score: 0.463
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8364999890327454     │
└───────────────────────────┴───────────────────────────┘